In [1]:
import pandas as pd

TEST = '/kaggle/input/playground-series-s4e4/test.csv'
TRAIN = '/kaggle/input/playground-series-s4e4/train.csv'
test_df = pd.read_csv(filepath_or_buffer=TEST, index_col=['id']).drop(columns=['Sex'])

train_df = pd.read_csv(filepath_or_buffer=TRAIN, index_col=['id']).drop(columns=['Sex'])
train_df.head()

,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
id,,,,,,,,
0,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9


In [2]:
import warnings
from plotly import express

warnings.filterwarnings(action='ignore', category=FutureWarning)
express.histogram(data_frame=train_df, x='Rings', )

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

COLUMNS = ['Length', 'Diameter', 'Height', 'Whole weight', 'Whole weight.1', 'Whole weight.2', 'Shell weight',]
TARGET = 'Rings'

X_train, X_test, y_train, y_test = train_test_split(train_df[COLUMNS], train_df[TARGET], test_size=0.2, random_state=2024)

Let's make a simple Keras regression model and be done.

In [4]:
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Input


def baseline_model():
    model = Sequential()
    model.add(Input((7, )))
    model.add(BatchNormalization())
    model.add(Dense(16, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.20))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.20))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.20))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.20))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.20))
    model.add(Dense(1, kernel_initializer='normal')) 
    model.compile(optimizer='adam', loss = 'mean_squared_error', metrics=['mae', 'mse']) 
    return model

2024-04-16 17:20:27.275349: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-16 17:20:27.275486: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-16 17:20:27.470528: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
model = baseline_model()
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ batch_normalization             │ (None, 7)              │            28 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,437 (21.24 KB)

 Trainable params: 5,423 (21.18 KB)

 Non-trainable params: 14 (56.00 B)

In [6]:
history = model.fit(X_train, y_train, epochs=100, verbose=1, validation_split = 0.2)

Epoch 1/100
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 26.5844 - mae: 3.6743 - mse: 26.5844 - val_loss: 3.9550 - val_mae: 1.2898 - val_mse: 3.9600
Epoch 2/100
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 6.9197 - mae: 1.9035 - mse: 6.9197 - val_loss: 3.7912 - val_mae: 1.3725 - val_mse: 3.7941
Epoch 3/100
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 6.2074 - mae: 1.7923 - mse: 6.2074 - val_loss: 3.9610 - val_mae: 1.2907 - val_mse: 3.9666
Epoch 4/100
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 5.8806 - mae: 1.7380 - mse: 5.8806 - val_loss: 3.8408 - val_mae: 1.3385 - val_mse: 3.8453
Epoch 5/100
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 5.3553 - mae: 1.6455 - mse: 5.3553 - val_loss: 3.7474 - val_mae: 1.3422 - val_mse: 3.7518
Epoch 6/100
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 5.2245 - mae: 1.6162 - mse: 5.2245 - val_loss: 3.8379 - val_mae: 1.3963 - val_mse: 3.8423
Epoch 7/100
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 5.1133 - mae: 

In [7]:
history_df = pd.DataFrame(data=history.history)
history_df['epoch'] = history.epoch
history_df.tail()

,loss,mae,mse,val_loss,val_mae,val_mse,epoch
95,4.367307,1.455554,4.367264,3.788980,1.355430,3.791694,95
96,4.369421,1.457484,4.370310,3.852340,1.335225,3.855975,96
97,4.378524,1.455232,4.375543,3.917813,1.340792,3.921915,97
98,4.323383,1.449414,4.323701,3.935740,1.355857,3.939825,98
99,4.389284,1.456198,4.389896,3.945315,1.340994,3.949472,99


In [8]:
from plotly import express
express.line(data_frame=history_df, x='epoch', y=['mae', 'val_mae']).show()
express.line(data_frame=history_df, x='epoch', y=['loss', 'val_loss']).show()

In [9]:
import numpy as np
best_df = pd.DataFrame(data={'id' : test_df.index.tolist(), 'Rings': model.predict(test_df[COLUMNS]).flatten()})
best_df.to_csv(path_or_buf='/kaggle/working/submission.csv', index=False)

1888/1888 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
